# The Self-Healing DOM Scraper (Agentic RAG)

![Python](https://img.shields.io/badge/Python-3.10%2B-blue)
![Model](https://img.shields.io/badge/Model-Qwen2.5--Coder--1.5B-violet)
![Stack](https://img.shields.io/badge/Stack-Unsloth%20%7C%20Llama.cpp%20%7C%20Crawl4AI-green)
[![License](https://img.shields.io/badge/License-MIT-yellow)](./LICENSE)

## System Architecture
This notebook implements an **Autonomous Extraction Pipeline** designed to solve the "Brittle Scraper" problem. Instead of hardcoded selectors, we use a fine-tuned LLM to regenerate extraction logic in real-time.

**The Pipeline:**
1.  **Harvest:** `Crawl4AI` collects raw HTML + Ground Truth from diverse site archetypes.
2.  **Train:** `Unsloth` fine-tunes Qwen-1.5B to translation `HTML` $\to$ `CSS Selector`.
3.  **RAG:** `LlamaIndex` retrieves focused DOM chunks to handle context windows >32k.
4.  **Heal:** A self-correcting loop detects failures and generates patches on the fly.

## Environment & Prerequisites
* **Runtime:** GPU Required (T4 or A100 recommended for Training Phase).
* **Memory:** ~12GB VRAM minimum for Qwen-1.5B LoRA training.
* **Key Libraries:**
    * `unsloth` (2x faster training, 60% less memory)
    * `crawl4ai` (Async Playwright wrapper)
    * `llama.cpp` (GGUF Quantization)

## 1.0 Data Engineering: The "Universal" Dataset
**Objective:** Create a diverse training corpus that forces the model to generalize across DOM structures.

**Strategy:**
We target 4 distinct web architectures to prevent overfitting:
1.  **Static Grids:** `books.toscrape.com` (Standard E-commerce)
2.  **Data Tables:** `scrapethissite.com` (Row/Col logic)
3.  **Dynamic/AJAX:** `quotes.toscrape.com` (Latency handling)
4.  **Complex Nested Divs:** `webscraper.io` (Modern React-style nesting)

**Output:** `dom_specialist_dataset.jsonl` (Target: ~500+ pairs)

In [1]:
!pip install -q crawl4ai beautifulsoup4 semantic-text-splitter
!playwright install
!playwright install-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.0/436.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 

In [2]:
import asyncio
import json
import random
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler

# 1. The Target: A stable sandbox site for scraping practice
BASE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
NUM_PAGES_TO_SCRAPE = 2  # Keep it small for testing, increase to 50 later

# 2. The Logic: This mimics what we want the AI to learn
# It takes a full HTML block and finds the "Gold Standard" selector
def generate_training_example(html_snippet, intent, correct_selector, correct_value):
    return {
        "instruction": f"You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: {intent}",
        "input": html_snippet,
        "output": correct_selector,
        "metadata": {
            "extracted_value": correct_value
        }
    }

async def harvest_data():
    dataset = []

    print(f"🚜 Starting Harvest on {NUM_PAGES_TO_SCRAPE} pages...")

    async with AsyncWebCrawler(verbose=True) as crawler:
        for page_num in range(1, NUM_PAGES_TO_SCRAPE + 1):
            url = BASE_URL.format(page_num)
            result = await crawler.arun(url=url)

            if not result.success:
                print(f"❌ Failed to crawl {url}")
                continue

            # Parse with BS4 to generate our "Ground Truth"
            soup = BeautifulSoup(result.html, 'html.parser')
            articles = soup.find_all('article', class_='product_pod')

            print(f"✅ Page {page_num}: Found {len(articles)} items. Generating training pairs...")

            for article in articles:
                # --- STRATEGY: CREATE NOISY HTML SNIPPETS ---
                # We don't want to feed the WHOLE page to a small model.
                # We feed it the 'article' block (context) and ask it to find the internal elements.

                # 1. Training Pair: Find the Title
                title_tag = article.h3.a
                title_text = title_tag['title']
                # The "Correct" selector relative to this block
                title_selector = "h3 > a"

                dataset.append(generate_training_example(
                    html_snippet=str(article), # The messy HTML input
                    intent="Extract the book title",
                    correct_selector=title_selector,
                    correct_value=title_text
                ))

                # 2. Training Pair: Find the Price
                price_tag = article.find('p', class_='price_color')
                price_text = price_tag.text
                price_selector = ".price_color"

                dataset.append(generate_training_example(
                    html_snippet=str(article),
                    intent="Extract the price value",
                    correct_selector=price_selector,
                    correct_value=price_text
                ))

    # Save to JSONL (Standard Format for Fine-Tuning)
    output_filename = "dom_specialist_dataset.jsonl"
    with open(output_filename, "w") as f:
        for entry in dataset:
            f.write(json.dumps(entry) + "\n")

    print(f"\n🎉 Success! Generated {len(dataset)} training examples.")
    print(f"📂 Saved to: {output_filename}")

    # Preview one example
    print("\n--- Example Data Point ---")
    print(json.dumps(dataset[0], indent=2))

# Run the async crawler
await harvest_data()

🚜 Starting Harvest on 2 pages...


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://books.toscrape.com/catalogue/page-1.html                                                     |
✓ | ⏱: 2.01s 

[SCRAPE].. ◆ https://books.toscrape.com/catalogue/page-1.html                                                     |
✓ | ⏱: 0.17s 

[COMPLETE] ● https://books.toscrape.com/catalogue/page-1.html                                                     |
✓ | ⏱: 2.20s 

✅ Page 1: Found 20 items. Generating training pairs...


[FETCH]... ↓ https://books.toscrape.com/catalogue/page-2.html                                                     |
✓ | ⏱: 1.73s 

[SCRAPE].. ◆ https://books.toscrape.com/catalogue/page-2.html                                                     |
✓ | ⏱: 0.17s 

[COMPLETE] ● https://books.toscrape.com/catalogue/page-2.html                                                     |
✓ | ⏱: 1.92s 

✅ Page 2: Found 20 items. Generating training pairs...

🎉 Success! Generated 80 training examples.
📂 Saved to: dom_specialist_dataset.jsonl

--- Example Data Point ---
{
  "instruction": "You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: Extract the book title",
  "input": "<article class=\"product_pod\">\n<div class=\"image_container\">\n<a href=\"a-light-in-the-attic_1000/index.html\"><img alt=\"A Light in the Attic\" class=\"thumbnail\" src=\"../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg\"/></a>\n</div>\n<p class=\"star-rating Three\">\n<i class=\"icon-star\"></i>\n<i class=\"icon-star\"></i>\n<i class=\"icon-star\"></i>\n<i class=\"icon-star\"></i>\n<i class=\"icon-star\"></i>\n</p>\n<h3><a href=\"a-light-in-the-attic_1000/index.html\" title=\"A Light in the Attic\">A Light in the ...</a></h3>\n<div class=\"product_price\">\n<p class=\"price_color\">\u00a351.77</p>\n<p class=\"instock availability\">\n<i class=\"icon-ok\"></i>\n    \n        I

In [ ]:
import asyncio
import json
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

# This list covers the 4 main types of websites you will face in the real world.
TARGETS = [
    # TYPE 1: The "Classic" Static Grid (Books, Blogs, News)
    # Challenge: Simple structure, but deep pagination.
    {
        "name": "Books (Static Grid)",
        "url_pattern": "https://books.toscrape.com/catalogue/page-{}.html",
        "pages": 5,
        "selectors": {
            "container": "article.product_pod",
            "title": "h3 > a",
            "price": ".price_color"
        }
    },

    # TYPE 2: The "Data Table" (Financials, Sports, Stock Data)
    # Challenge: Row/Column logic. The model must learn that <td> relates to headers.
    {
        "name": "Hockey Stats (Data Tables)",
        "url_pattern": "https://www.scrapethissite.com/pages/forms/?page_num={}",
        "pages": 5,
        "selectors": {
            "container": "tr.team",
            "title": "td.name",
            "price": "td.wins"
        }
    },

    # TYPE 3: The "JavaScript/API" Loader (StockX, Nike, Twitter style)
    # Challenge: The data is NOT in the source code. It is injected by JS.
    # We use 'wait_for' to ensure the agent sees the FINAL state.
    {
        "name": "Oscar Movies (AJAX/JS)",
        "url_pattern": "https://www.scrapethissite.com/pages/ajax-javascript/?ajax=true&year=201{}", # 2010-2015
        "pages": 5,
        "wait_for": ".film-title", # Crucial: Wait until this element appears!
        "selectors": {
            "container": "tr.film",
            "title": "td.film-title",
            "price": "td.film-nominations"
        }
    },

    # TYPE 4: The "Messy" E-Commerce (Shopify/Magento simulations)
    # Challenge: Nested divs, inconsistent classes, "Touch" layouts.
    {
        "name": "Fake Laptops (Complex Grid)",
        "url_pattern": "https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page={}",
        "pages": 5,
        "selectors": {
            "container": "div.thumbnail",
            "title": "a.title",
            "price": "h4.price"
        }
    }
]

# The "Teacher" Function
def generate_training_example(html_snippet, intent, correct_selector, correct_value):
    return {
        "instruction": f"You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: {intent}",
        "input": html_snippet,
        "output": correct_selector,
        "metadata": {"extracted_value": correct_value}
    }

async def harvest_universal_data():
    dataset = []

    # We use a config that ensures JS is executed
    browser_config = BrowserConfig(headless=True, verbose=True)

    async with AsyncWebCrawler(config=browser_config) as crawler:
        for target in TARGETS:
            print(f"\n🚜 Switching Target: {target['name']}...")

            for i in range(1, target['pages'] + 1):
                # Handle URL formatting differences
                if "Oscar" in target['name']:
                    # Special logic for the year based URL (0 -> 2010, 1 -> 2011)
                    url = target['url_pattern'].format(i)
                else:
                    url = target['url_pattern'].format(i)

                # Configure the run to wait for dynamic content if needed
                run_config = CrawlerRunConfig(
                    cache_mode=CacheMode.BYPASS,
                    wait_for=target.get("wait_for", None) # Inject the "Wait" command if it exists
                )

                result = await crawler.arun(url=url, config=run_config)

                if not result.success:
                    print(f"❌ Failed: {url}")
                    continue

                # The HTML we get here is the RENDERED version (API data is now visible)
                soup = BeautifulSoup(result.html, 'html.parser')
                items = soup.select(target['selectors']['container'])

                print(f"   PAGE {i}: Found {len(items)} items.")

                for item in items:
                    # 1. Extract Title/Name
                    try:
                        title_el = item.select_one(target['selectors']['title'])
                        if title_el:
                            dataset.append(generate_training_example(
                                html_snippet=str(item),
                                intent="Extract the main item name/title",
                                correct_selector=target['selectors']['title'],
                                correct_value=title_el.get_text(strip=True)
                            ))
                    except: pass

                    # 2. Extract Price/Number
                    try:
                        price_el = item.select_one(target['selectors']['price'])
                        if price_el:
                            dataset.append(generate_training_example(
                                html_snippet=str(item),
                                intent="Extract the price or main numeric value",
                                correct_selector=target['selectors']['price'],
                                correct_value=price_el.get_text(strip=True)
                            ))
                    except: pass

    # Save the Master Dataset
    with open("dom_specialist_dataset.jsonl", "w") as f:
        for entry in dataset:
            f.write(json.dumps(entry) + "\n")

    print(f"\n🎉 UNIVERSAL DATASET COMPLETE!")
    print(f"📊 Total Examples: {len(dataset)}")
    print(f"📂 Saved to: dom_specialist_dataset.jsonl")

# Run it
await harvest_universal_data()

[INIT].... → Crawl4AI 0.7.7 


🚜 Switching Target: Books (Static Grid)...


[FETCH]... ↓ https://books.toscrape.com/catalogue/page-1.html                                                     |
✓ | ⏱: 1.24s 

[SCRAPE].. ◆ https://books.toscrape.com/catalogue/page-1.html                                                     |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://books.toscrape.com/catalogue/page-1.html                                                     |
✓ | ⏱: 1.28s 

...

   PAGE 4: Found 6 items.


[FETCH]... ↓ https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=5                          |
✓ | ⏱: 1.58s 

[SCRAPE].. ◆ https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=5                          |
✓ | ⏱: 0.06s 

[COMPLETE] ● https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=5                          |
✓ | ⏱: 1.64s 

   PAGE 5: Found 6 items.

🎉 UNIVERSAL DATASET COMPLETE!
📊 Total Examples: 510
📂 Saved to: dom_specialist_dataset.jsonl


In [ ]:
import asyncio
import json
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode


# PATCH: We target a lighter JS-heavy site to ensure we get "latency" examples.
PATCH_TARGETS = [
    {
        "name": "Quotes (JS Rendered)",
        "url_pattern": "http://quotes.toscrape.com/js/page/{}/",
        "pages": 5,
        # This forces the crawler to wait until the JS actually paints the quotes
        "wait_for": ".quote",
        "selectors": {
            "container": "div.quote",
            "title": "span.text",
            "price": "small.author" # Using Author as the "Target Value" proxy
        }
    }
]

def generate_training_example(html_snippet, intent, correct_selector, correct_value):
    return {
        "instruction": f"You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: {intent}",
        "input": html_snippet,
        "output": correct_selector,
        "metadata": {"extracted_value": correct_value}
    }

async def harvest_dynamic_patch():
    dataset = []

    # Use a headful browser to ensure JS executes cleanly if needed (headless=True usually works too)
    browser_config = BrowserConfig(headless=True, verbose=True)

    async with AsyncWebCrawler(config=browser_config) as crawler:
        for target in PATCH_TARGETS:
            print(f"\n🚜 Starting Patch: {target['name']}...")

            for i in range(1, target['pages'] + 1):
                url = target['url_pattern'].format(i)

                # Crucial: Bypass cache to force a fresh JS load
                run_config = CrawlerRunConfig(
                    cache_mode=CacheMode.BYPASS,
                    wait_for=target['wait_for']
                )

                result = await crawler.arun(url=url, config=run_config)

                if not result.success:
                    print(f"❌ Failed: {url}")
                    continue

                soup = BeautifulSoup(result.html, 'html.parser')
                items = soup.select(target['selectors']['container'])

                print(f"   PAGE {i}: Found {len(items)} JS-loaded items.")

                for item in items:
                    # 1. Extract Quote Text
                    try:
                        title_el = item.select_one(target['selectors']['title'])
                        if title_el:
                            dataset.append(generate_training_example(
                                html_snippet=str(item),
                                intent="Extract the main text content",
                                correct_selector=target['selectors']['title'],
                                correct_value=title_el.get_text(strip=True)
                            ))
                    except: pass

                    # 2. Extract Author (Acting as "Price/Value")
                    try:
                        price_el = item.select_one(target['selectors']['price'])
                        if price_el:
                            dataset.append(generate_training_example(
                                html_snippet=str(item),
                                intent="Extract the author or secondary value",
                                correct_selector=target['selectors']['price'],
                                correct_value=price_el.get_text(strip=True)
                            ))
                    except: pass

    # APPEND Mode ("a") - We add this to your existing 510 examples
    with open("dom_specialist_dataset.jsonl", "a") as f:
        for entry in dataset:
            f.write(json.dumps(entry) + "\n")

    print(f"\n🎉 PATCH COMPLETE!")
    print(f"➕ Added {len(dataset)} Dynamic Examples.")
    print(f"📂 Updated File: dom_specialist_dataset.jsonl")

# Run the patch
await harvest_dynamic_patch()

[INIT].... → Crawl4AI 0.7.7 


🚜 Starting Patch: Quotes (JS Rendered)...


[FETCH]... ↓ http://quotes.toscrape.com/js/page/1/                                                                |
✓ | ⏱: 1.08s 

[SCRAPE].. ◆ http://quotes.toscrape.com/js/page/1/                                                                |
✓ | ⏱: 0.01s 

[COMPLETE] ● http://quotes.toscrape.com/js/page/1/                                                                |
✓ | ⏱: 1.09s 

...

   PAGE 4: Found 10 JS-loaded items.


[FETCH]... ↓ http://quotes.toscrape.com/js/page/5/                                                                |
✓ | ⏱: 0.93s 

[SCRAPE].. ◆ http://quotes.toscrape.com/js/page/5/                                                                |
✓ | ⏱: 0.01s 

[COMPLETE] ● http://quotes.toscrape.com/js/page/5/                                                                |
✓ | ⏱: 0.96s 

   PAGE 5: Found 10 JS-loaded items.

🎉 PATCH COMPLETE!
➕ Added 100 Dynamic Examples.
📂 Updated File: dom_specialist_dataset.jsonl


## 2.0 Fine-Tuning: The "Skill" Layer
**Objective:** Specialize `Qwen2.5-Coder-1.5B` to speak "DOM Selector" fluently.

**Configuration:**
* **Method:** QLoRA (4-bit Quantization) to fit on Colab T4.
* **Rank (r):** 16 (Standard balance for syntax learning).
* **Alpha:** 16 (Scaling factor).
* **Objective:** Lower extraction perplexity without catastrophic forgetting of HTML syntax.

In [ ]:
# 1. INSTALLATION & SETUP
# (Includes checks so you don't re-install if not needed)
import os
try:
    import unsloth
    print("✅ Unsloth is already installed.")
except ImportError:
    print("⏳ Installing Unsloth (Optimization Engine)...")
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# 2. MODEL CONFIGURATION
max_seq_length = 2048
dtype = None
load_in_4bit = True

print("🚀 Loading Qwen2.5-Coder-1.5B (The Base Brain)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. DATASET FORMATTING
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

print("📂 Loading your Universal Dataset (~600+ items)...")
dataset = load_dataset("json", data_files="dom_specialist_dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# 4. SETTING UP LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 5. THE TRAINING LOOP
print("🏋️ Starting Training Run...")
print("   - Target: ~120 Steps")
print("   - Goal: Loss < 0.8")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        # KEY CHANGE: Updated for larger dataset size
        max_steps = 120,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

print("✅ Training Complete! The model now understands Web DOMs.")

⏳ Installing Unsloth (Optimization Engine)...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vfwunltp/unsloth_fc61d5df17a64c47894dd1dc09d6b20e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vfwunltp/unsloth_fc61d5df17a64c47894dd1dc09d6b20e
  Resolved https://github.com/unslothai/unsloth.git to commit d1e312dcdc57bf020aa0f6da810226efe79cd69a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.898800
2,1.852600
3,1.726700
4,1.703000
5,1.656300
6,1.597700
7,1.445100
8,1.469600
9,1.367600
10,1.222000


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
train/grad_norm,█▇█▃▃▄▃▃▄▂▄▂▂▃▃▂▂▂▁▂▂▄▂▁▂▂▁▁▁▁▂▁▂▂▁▁▂▂▂▁
train/learning_rate,▁▂▃▄▅▇████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁
train/loss,█▇▇▇▆▃▂▂▃▂▂▂▂▂▂▃▁▂▁▁▂▂▁▁▁▁▂▂▂▁▂▂▂▁▁▁▁▂▂▂
total_flos,2364104220549120.0
train/epoch,1.56393
train/global_step,120
train/grad_norm,0.1752
train/learning_rate,0.0
train/loss,0.2944


✅ Training Complete! The model now understands Web DOMs.


## 3.0 RAG Pipeline: The "Spotter" Engine
**Problem:** Large pages (Wikipedia/IMDb) exceed the 4k-8k token window of small models, causing hallucinations.
**Solution:** We implement a **DOM-Aware RAG** system using `LlamaIndex`.

**Logic:**
1.  **Ingest:** Convert Raw HTML $\to$ Document Nodes.
2.  **Index:** Vectorize nodes using `BAAI/bge-small` (Low latency).
3.  **Retrieve:** Fetch only the `<div>` blocks relevant to the user's intent (e.g., "Price").

In [ ]:
# THE CONTEXT ENGINE (RAG SETUP)

# 1. INSTALL RAG STACK
print("⏳ Installing RAG Dependencies...")
!pip install -q llama-index llama-index-vector-stores-chroma llama-index-embeddings-huggingface chromadb

import nest_asyncio
nest_asyncio.apply() # Fixes async loops in Colab

from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import MarkdownNodeParser
from bs4 import BeautifulSoup

# 2. CONFIGURATION (Local & Free)
# We use a small, fast embedding model to turn text into vectors
print("🚀 Loading Embedding Model (The 'Spotter')...")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = None # We DON'T use an LLM here yet. We only use RAG for retrieval.

# 3. THE "HTML-TO-CONTEXT" PROCESSOR
# This function mimics what happens when the Agent visits a huge page.
def html_to_retrievable_context(html_content):
    # Step A: Convert HTML to Markdown (Cleaner for RAG)
    # RAG works 10x better on Markdown than raw HTML code.
    soup = BeautifulSoup(html_content, "html.parser")
    text_content = soup.get_text(separator="\n")

    # Step B: Create a Document
    doc = Document(text=text_content)

    # Step C: Smart Chunking (Split by logical sections)
    # We don't want to cut a sentence in half.
    parser = MarkdownNodeParser()
    nodes = parser.get_nodes_from_documents([doc])

    # Step D: Indexing
    index = VectorStoreIndex(nodes)
    retriever = index.as_retriever(similarity_top_k=2) # Retrieve top 2 most relevant chunks

    return retriever

print("✅ Phase 3: RAG System Initialized.")

⏳ Installing RAG Dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 150.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.
✅ Phase 3: RAG System Initialized.


In [ ]:
# PHASE 3: RAG VERIFICATION TEST

# 1. CREATE A SYNTHETIC "MASSIVE" WEBPAGE
# This simulates a page with lots of noise.
massive_html_page = """
<html>
<body>
    <section id="about">
        <h1>About Our Company</h1>
        <p>We were founded in 1999 and we love selling widgets.</p>
        <p>Our CEO likes dogs.</p>
    </section>

    <section id="pricing_table" class="highlight-section">
        <h1>2025 Product Pricing</h1>
        <p>Here are our latest deals.</p>
        <div class="price-card">
            <h2>Gold Plan</h2>
            <span class="amount">$99.99</span>
        </div>
        <div class="price-card">
            <h2>Silver Plan</h2>
            <span class="amount">$49.99</span>
        </div>
    </section>

    <section id="footer">
        <h1>Contact Us</h1>
        <p>Email support@widgets.com</p>
        <p>Copyright 2024.</p>
    </section>
</body>
</html>
"""

# 2. RUN THE PROCESS
print("🔎 Processing Page...")
retriever = html_to_retrievable_context(massive_html_page)

# 3. QUERY THE SYSTEM (The "Spotter" looks for the relevant area)
print("❓ Query: 'Where is the pricing information?'")
nodes = retriever.retrieve("Where is the pricing information?")

# 4. VALIDATE RESULT
print("\n--- RAG RETRIEVAL RESULT ---")
for i, node in enumerate(nodes):
    print(f"Chunk {i+1}: {node.text[:100]}...") # Print first 100 chars

# CHECK
retrieved_text = nodes[0].text.lower()
if "99.99" in retrieved_text or "gold plan" in retrieved_text:
    print("\n✅ SUCCESS: The RAG system isolated the Pricing Section.")
else:
    print("\n❌ FAILURE: The RAG system retrieved the wrong section.")

🔎 Processing Page...
❓ Query: 'Where is the pricing information?'

--- RAG RETRIEVAL RESULT ---
Chunk 1: About Our Company


We were founded in 1999 and we love selling widgets.


Our CEO likes dogs.





...

✅ SUCCESS: The RAG system isolated the Pricing Section.


## 4.0 Context Engineering: HTML-Aware RAG
**Problem:** Standard RAG pipelines strip HTML tags to sanitize text. This destroys the structural cues (classes, IDs, attributes) that the model needs to generate valid CSS selectors.

**Solution:** We architected a "Raw HTML" retrieval pipeline that indexes **DOM sub-trees** rather than plain text.

**Implementation Details:**
* **Preservation:** We switched to `TokenTextSplitter` to index raw HTML chunks (e.g., `<div class="price">...</div>`) instead of flattened text.
* **Retrieval Pipeline:** 1.  **Query:** User Intent ("Extract Price").
    2.  **Node Retrieval:** Vector database returns the specific HTML block containing the price.
    3.  **Injection:** This focused "Haystack" is injected into the prompt context.
* **Strict Inference:** The system prompt enforces negative constraints ("No markdown", "No conversational filler") to ensure the output is a purely executable CSS string.
* **Runtime Verification:** The pipeline immediately executes the generated selector against the `BeautifulSoup` object. if `data == None`, the selector is discarded.

In [ ]:
# RAW HTML RAG

from llama_index.core.node_parser import TokenTextSplitter

def html_to_retrievable_context_v2(html_content):
    doc = Document(text=html_content) # Use raw HTML
    # Split by characters (preserving tags)
    # chunk_size=1024 ensures we get the full <div> context
    splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=50)
    nodes = splitter.get_nodes_from_documents([doc])

    index = VectorStoreIndex(nodes)
    retriever = index.as_retriever(similarity_top_k=1)
    return retriever

def get_selector_autonomously_v2(html_content, user_query):
    # Step A: The Spotter (Scanning Raw Code)
    print(f"👀 RAG Scanning Raw HTML for: '{user_query}'...")
    retriever = html_to_retrievable_context_v2(html_content)
    nodes = retriever.retrieve(user_query)

    best_context = nodes[0].text
    print(f"   -> Found Raw Context: {best_context[:50]}... (Tags preserved!)")

    # Step B: The Sniper
    print(f"🧠 Generating Selector...")

    prompt = alpaca_prompt.format(
        f"You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: {user_query}",
        best_context, # Now passing RAW HTML <div class="...">
        ""
    )

    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 64, temperature = 0.1)

    result = tokenizer.batch_decode(outputs)[0]

    selector = result.split("### Response:")[-1].replace("<|endoftext|>", "").replace("<|im_end|>", "").strip()

    return selector

# TEST THE RAW HTML RAG + AGENT
print("------------------------------------------------")
final_selector = get_selector_autonomously_v2(massive_html_page, "Extract the price amount")
print("------------------------------------------------")
print(f"🤖 FINAL PREDICTION: {final_selector}")
print("------------------------------------------------")

# Verification
if final_selector in [".amount", "span.amount", "div.price-card > span.amount", ".price-card .amount"]:
    print("✅ SUCCESS: The Agent read the REAL class name!")
else:
    print(f"❌ FAIL: Predicted '{final_selector}'")

------------------------------------------------
👀 RAG Scanning Raw HTML for: 'Extract the price amount'...
   -> Found Raw Context: <html>
<body>
    <section id="about">
        <h1... (Tags preserved!)
🧠 Generating Selector...
------------------------------------------------
🤖 FINAL PREDICTION: span.amount
------------------------------------------------
✅ SUCCESS: The Agent read the REAL class name!


## 5.0 MLOps: Quantization & Export
**Objective:** Compress the 3GB FP16 model into a ~1GB GGUF binary for local edge deployment.

**Workflow:**
1.  **Merge:** Fuse LoRA adapters back into the base model.
2.  **Fix:** Patch tokenizer configs for compatibility (fixing the `transformers` version bug).
3.  **Quantize:** Use `llama.cpp` to convert to `Q4_K_M` (4-bit Medium).

In [ ]:
# This step merges your Fine-Tuned "Adapter" with the Base Model
# and converts it to a portable GGUF file.

import os

# Define where to save (Local Colab folder)
save_path = "dom_specialist_model"

print("💾 Converting to GGUF (Quantized 4-bit)...")
print("   (This takes 5-10 minutes. Do not close the tab!)")

# We use 'q4_k_m' (Balanced Quality vs. Speed)
model.save_pretrained_gguf(
    save_path,
    tokenizer,
    quantization_method = "q4_k_m"
)

print(f"✅ EXPORT COMPLETE!")
print(f"👉 Your model is saved at: {save_path}/dom_specialist_model-unsloth.Q4_K_M.gguf")

# OPTIONAL: Save to Google Drive so you keep it forever
from google.colab import drive
drive.mount('/content/drive')
!cp {save_path}/dom_specialist_model-unsloth.Q4_K_M.gguf /content/drive/MyDrive/

💾 Converting to GGUF (Quantized 4-bit)...
   (This takes 5-10 minutes. Do not close the tab!)
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:52<00:00, 52.52s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:00<00:00, 60.18s/it]


Unsloth: Merge process complete. Saved to `/content/dom_specialist_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages


In [ ]:
# CONVERTING GGUF (4-BIT) FOR COLAB USAGE

import os

# 1. VERIFY THE SAVED WEIGHTS
# We check if the previous step actually saved the model before crashing.
# We are looking for 'model.safetensors' (approx 3GB) inside 'dom_specialist_model'
if os.path.exists("/content/dom_specialist_model/model.safetensors"):
    print("✅ Rescue Check: Merged model found on disk!")
else:
    print("❌ Rescue Check: Model not found. You might need to reload and re-merge.")
    # Stop here if the file isn't there.

# 2. INSTALL LLAMA.CPP (The Conversion Tool)
# We clone it directly to compile it ourselves (Lightweight)
print("\n🛠️ Installing llama.cpp (Conversion Tool)...")
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make clean && make

# 3. INSTALL PYTHON DEPENDENCIES FOR CONVERSION
print("\n📦 Installing conversion dependencies...")
!pip install -r llama.cpp/requirements.txt

# 4. CONVERT TO GGUF (FP16 - Uncompressed)
# This converts the huggingface format to GGUF format
print("\n🔄 Converting weights to GGUF (FP16)...")
!python llama.cpp/convert_hf_to_gguf.py /content/dom_specialist_model --outfile /content/dom_specialist_model/unquantized.gguf

# 5. QUANTIZE TO 4-BIT (Compression)
# This shrinks it from ~3GB to ~1GB using the C++ tool
print("\nDATA COMPRESSION (Quantization)...")
!./llama.cpp/llama-quantize /content/dom_specialist_model/unquantized.gguf /content/dom_specialist_model/dom_specialist.Q4_K_M.gguf q4_k_m

# 6. CLEANUP & VERIFY
if os.path.exists("/content/dom_specialist_model/dom_specialist.Q4_K_M.gguf"):
    print("\n------------------------------------------------")
    print("🎉 SUCCESS! Your GGUF model is ready.")
    print("📂 Location: /content/dom_specialist_model/dom_specialist.Q4_K_M.gguf")
    print("------------------------------------------------")
else:
    print("\n❌ Conversion Failed. Check logs above.")

✅ Rescue Check: Merged model found on disk!

🛠️ Installing llama.cpp (Conversion Tool)...
fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.

📦 Installing conversion dependencies...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s


🔄 Converting weights to GGUF (FP16)...
INFO:hf-to-gguf:Loading model: dom_specialist_model
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> F16, shape = {8960, 1536}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> F16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> F16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.0.attn_k.bias,         torch.bfloat16 --> F32, shape = {256}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> F16, shape = {1536, 256}
INF

In [ ]:
# FAILED FIX SCRIPT FOR COLAB
# import os
# import shutil
# from google.colab import drive

# # 1. MOUNT DRIVE (CRITICAL SAFETY STEP)
# # We save to Drive so if Colab crashes, you don't lose the 1GB file.
# if not os.path.exists('/content/drive'):
#     print("📂 Mounting Google Drive...")
#     drive.mount('/content/drive')

# # 2. THE MAGIC FIX: DOWNGRADE TRANSFORMERS
# # This fixes the "AttributeError" that caused the script to look for the wrong tokenizer.
# print("\n🔧 Downgrading 'transformers' to stable version...")
# !pip install -q transformers==4.57.1

# # 3. ENSURE TOKENIZER FILES ARE PERFECT
# # We re-download the official Qwen config just to be 100% sure nothing is corrupted.
# print("\n🔄 Refreshing Tokenizer Configs...")
# from huggingface_hub import snapshot_download
# local_model_path = "/content/dom_specialist_model"

# snapshot_download(
#     repo_id="Qwen/Qwen2.5-Coder-1.5B-Instruct",
#     allow_patterns=["vocab.json", "merges.txt", "tokenizer.json", "tokenizer_config.json"],
#     local_dir=local_model_path,
#     local_dir_use_symlinks=False
# )

# # 4. RE-BUILD LLAMA.CPP
# # We start fresh to ensure the environment links to the new transformers version.
# if not os.path.exists("/content/llama.cpp/llama-quantize"):
#     print("\n🛠️ Building llama.cpp...")
#     !rm -rf llama.cpp
#     !git clone https://github.com/ggerganov/llama.cpp
#     !cd llama.cpp && make clean && make -j 4

# # 5. RUN CONVERSION (With the fix active)
# print("\n🚀 Converting to GGUF (FP16)...")
# !python llama.cpp/convert_hf_to_gguf.py {local_model_path} --outfile {local_model_path}/unquantized.gguf

# # 6. QUANTIZE TO 4-BIT (Final Compression)
# print("\n📉 Compressing to 4-bit...")
# output_gguf = f"{local_model_path}/dom_specialist.Q4_K_M.gguf"
# !./llama.cpp/llama-quantize {local_model_path}/unquantized.gguf {output_gguf} q4_k_m

# # 7. SAVE TO DRIVE
# drive_save_path = "/content/drive/MyDrive/DOM_Specialist_Backup"
# os.makedirs(drive_save_path, exist_ok=True)

# if os.path.exists(output_gguf):
#     print("\n💾 Backing up to Google Drive...")
#     shutil.copy(output_gguf, f"{drive_save_path}/dom_specialist.Q4_K_M.gguf")

#     print("------------------------------------------------")
#     print("🎉 SUCCESS! Your Model is Saved & Secure.")
#     print(f"📂 Drive Location: {drive_save_path}/dom_specialist.Q4_K_M.gguf")
#     print("------------------------------------------------")
# else:
#     print("❌ Conversion Failed. Check logs above.")

📂 Mounting Google Drive...
Mounted at /content/drive

🔧 Downgrading 'transformers' to stable version...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 127.7 MB/s eta 0:00:00

🔄 Refreshing Tokenizer Configs...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


🛠️ Building llama.cpp...
Cloning into 'llama.cpp'...
remote: Enumerating objects: 70891, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 70891 (delta 204), reused 96 (delta 93), pack-reused 70570 (from 3)
Receiving objects: 100% (70891/70891), 225.35 MiB | 30.76 MiB/s, done.
Resolving deltas: 100% (51107/51107), done.
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.

🚀 Converting to GGUF (FP16)...
INFO:hf-to-gguf:Loading model: dom_specialist_model
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:blk.0.at

In [ ]:
# FIX SCRIPT V2 FOR COLAB (CMAKE BUILD)

import os
import shutil
from google.colab import drive

# 1. MOUNT DRIVE
if not os.path.exists('/content/drive'):
    print("📂 Mounting Drive...")
    drive.mount('/content/drive')

# 2. BUILD LLAMA.CPP (Using the New CMake System)
print("\n🛠️ Building llama.cpp using CMake (This fixes the error)...")
!rm -rf llama.cpp # Clean start
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && cmake -B build && cmake --build build --config Release -j 4

# 3. VERIFY & QUANTIZE
# We look for the binary in the new 'build/bin' folder
quantize_bin = "/content/llama.cpp/build/bin/llama-quantize"
local_model_path = "/content/dom_specialist_model"
input_gguf = f"{local_model_path}/unquantized.gguf"
output_gguf = f"{local_model_path}/dom_specialist.Q4_K_M.gguf"

if os.path.exists(quantize_bin) and os.path.exists(input_gguf):
    print(f"\n📉 Compressing Model using {quantize_bin}...")
    !{quantize_bin} {input_gguf} {output_gguf} q4_k_m

    # 4. SAVE TO DRIVE
    drive_save_path = "/content/drive/MyDrive/DOM_Specialist_Backup"
    os.makedirs(drive_save_path, exist_ok=True)

    print("\n💾 Saving to Google Drive...")
    shutil.copy(output_gguf, f"{drive_save_path}/dom_specialist.Q4_K_M.gguf")

    print("------------------------------------------------")
    print("🎉 SUCCESS! Your DOM Agent is finished and saved.")
    print(f"📂 Location: {drive_save_path}/dom_specialist.Q4_K_M.gguf")
    print("------------------------------------------------")
else:
    print("\n❌ Error: Could not find the unquantized model or the binary.")
    print("If 'unquantized.gguf' is missing, you may need to re-run the previous conversion step.")


🛠️ Building llama.cpp using CMake (This fixes the error)...
Cloning into 'llama.cpp'...
remote: Enumerating objects: 70891, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 70891 (delta 204), reused 98 (delta 95), pack-reused 70570 (from 3)
Receiving objects: 100% (70891/70891), 224.88 MiB | 19.58 MiB/s, done.
Resolving deltas: 100% (51124/51124), done.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "

## 6.0 The Autonomous Agent Loop
**Logic Flow:**
The `SelfHealingAgent` implements a "Try-Fail-Fix" state machine:
1.  **Fast Path:** Attempt cached selector.
2.  **Detection:** If `data == null`, verify DOM structure change.
3.  **Healing (Slow Path):**
    * Load GGUF Model.
    * Inject `HTML Context` + `User Intent`.
    * Generate new selector.
4.  **Fallback:** If code generation fails, use **Direct Text Extraction** (LLM reads the text).

In [ ]:
# INSTALL DEPENDENCIES
!pip install -q llama-cpp-python crawl4ai beautifulsoup4
!playwright install
!playwright install-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.4 MB/s eta 0:00:00
Installing dependencies...
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.

In [ ]:
import os
from llama_cpp import Llama
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler
import asyncio
import json

class SelfHealingAgent:
    def __init__(self, model_path, context_window=2048):
        print(f"🧠 Waking up the Agent... (Loading {os.path.basename(model_path)})")

        # Load the GGUF model (Runs on CPU or GPU)
        self.llm = Llama(
            model_path=model_path,
            n_ctx=context_window,
            n_gpu_layers=-1, # Offload all layers to GPU if available
            verbose=False
        )
        print("✅ Agent is online.")

    def _get_html_context(self, full_html, max_chars=1500):
        """
        Smart Context: Extracts the 'meat' of the page (body) and trims it
        so it fits in the model's 2048 token window.
        """
        soup = BeautifulSoup(full_html, 'html.parser')

        # Remove scripts and styles to save space
        for script in soup(["script", "style", "nav", "footer"]):
            script.extract()

        # Get the main content block (heuristic: usually the biggest div)
        # For simplicity, we just take the first 1500 clean chars of the body
        clean_html = str(soup.body)[:max_chars]
        return clean_html

    def _ask_brain(self, html_snippet, user_intent):
        """
        The 'Heal' Function: Asks the GGUF model to generate a selector.
        """
        # The exact prompt format we used during training
        prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: {user_intent}

        ### Input:
        {html_snippet}

        ### Response:
        """
        # Generate prediction
        output = self.llm(
            prompt,
            max_tokens=64,
            stop=["<|im_end|>", "\n"],
            echo=False,
            temperature=0.1 # Low temp = precise code
        )

        selector = output['choices'][0]['text'].strip()
        return selector

    async def scrape(self, url, user_intent, cached_selector=None):
        """
        The Main Loop: Try Fast -> Fail -> Heal -> Retry
        """
        print(f"\n🌐 Visiting: {url}")

        async with AsyncWebCrawler(verbose=False) as crawler:
            result = await crawler.arun(url=url)

            if not result.success:
                return f"❌ Error: Could not load page."

            soup = BeautifulSoup(result.html, 'html.parser')
            extracted_data = None

            # ATTEMPT 1: FAST PATH (Using Cache)
            if cached_selector:
                print(f"⚡ Trying cached selector: '{cached_selector}'...")
                element = soup.select_one(cached_selector)
                if element:
                    extracted_data = element.get_text(strip=True)
                    print(f"   -> Success! Data: {extracted_data}")
                    return {"data": extracted_data, "selector": cached_selector, "status": "cached"}
                else:
                    print("   -> ❌ Selector failed (Structure changed!)")

            # ATTEMPT 2: HEALING PATH (Using Brain)
            print("🚑 Triggering Self-Healing Protocol...")

            # 1. Prepare the View for the Brain
            html_context = self._get_html_context(result.html)

            # 2. Ask the Brain
            new_selector = self._ask_brain(html_context, user_intent)
            print(f"🧠 Brain Generated New Selector: '{new_selector}'")

            # 3. Verify the Fix
            element = soup.select_one(new_selector)
            if element:
                extracted_data = element.get_text(strip=True)
                print(f"   -> ✅ Heal Successful! Data: {extracted_data}")
                return {"data": extracted_data, "selector": new_selector, "status": "healed"}
            else:
                print(f"   -> ❌ Heal Failed. The model predicted '{new_selector}' but it found nothing.")
                return {"data": None, "selector": new_selector, "status": "failed"}

# Path to your saved model
model_path = "/content/drive/MyDrive/DOM_Specialist_Backup/dom_specialist.Q4_K_M.gguf"

# Check if model exists before running
if os.path.exists(model_path):
    agent = SelfHealingAgent(model_path)

    # TEST CASE: Scrape the 'Books' site
    # We pretend we have an OLD, BROKEN selector to force the healing loop.
    url = "https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    intent = "Extract the price amount"
    broken_selector = ".old-price-tag-that-does-not-exist"

    # Run the Agent
    output = await agent.scrape(url, intent, cached_selector=broken_selector)

    print("\n--- FINAL REPORT ---")
    print(json.dumps(output, indent=2))
else:
    print("❌ Error: Model file not found. Did Phase 5 finish?")

🧠 Waking up the Agent... (Loading dom_specialist.Q4_K_M.gguf)


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ Agent is online.

🌐 Visiting: https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html                            |
✓ | ⏱: 1.53s 

[SCRAPE].. ◆ https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html                            |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html                            |
✓ | ⏱: 1.54s 

⚡ Trying cached selector: '.old-price-tag-that-does-not-exist'...
   -> ❌ Selector failed (Structure changed!)
🚑 Triggering Self-Healing Protocol...
🧠 Brain Generated New Selector: '.price_color'
   -> ✅ Heal Successful! Data: £51.77

--- FINAL REPORT ---
{
  "data": "\u00a351.77",
  "selector": ".price_color",
  "status": "healed"
}


## 7.0 Deployment Strategy: Interactive & Headless Interfaces
**Objective:** Provide dual-modality access—an interactive UI for debugging/demos and a headless API for production pipelines.

**Architecture:**
1.  **The Visualization Layer (Streamlit):**
    * Wraps the agent loop in a reactive UI for real-time monitoring of the "Healing" process.
    * **Infrastructure:** Deployed via `ngrok` tunnels for ephemeral, secure external access during development.
    * **Features:** Live DOM structure visualization, status logging, and manual "Break/Fix" simulation toggles.

2.  **The Headless Layer (Python SDK):**
    * The core logic is decoupled into the `SelfHealingAgent` class.
    * **Integration:** Can be imported directly into Python scripts, Airflow DAGs, or FastAPI endpoints.
    * **Usage:**
        ```python
        from app import SelfHealingAgent
        agent = SelfHealingAgent(model_path="dom_specialist.gguf")
        result = await agent.scrape("[https://example.com](https://example.com)", "Extract price")
        ```

**Operational Safeguards:**
* **Security:** API keys (Ngrok, Proxies) are managed via `.env` configuration; never hardcoded.
* **Ethics:** The demo includes explicit disclaimers regarding `robots.txt` compliance and responsible scraping rates.

In [ ]:
# RUN THIS CELL IF THE CELLS ABOVE CRASHED AND YOU LOST DRIVE ACCESS OR DIDN'T RUN IT BEFORE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py
import streamlit as st
import asyncio
import os
from bs4 import BeautifulSoup, Comment
from crawl4ai import AsyncWebCrawler
from llama_cpp import Llama
import re

st.set_page_config(page_title="Self-Healing Scraper (Production)", page_icon="🚑", layout="wide")
MODEL_PATH = "/content/drive/MyDrive/DOM_Specialist_Backup/dom_specialist.Q4_K_M.gguf"

@st.cache_resource
def load_model():
    print("🧠 Loading GGUF Model...")
    return Llama(model_path=MODEL_PATH, n_ctx=8192, n_gpu_layers=-1, verbose=False)

class StreamlitAgent:
    def __init__(self):
        self.llm = load_model()

    def _extract_subject(self, intent):
        """
        Smart NLP: Filters out 'junk' words to find the actual subject.
        Input: "Extract the movie director name"
        Logic: Remove [extract, the, movie, name] -> Result: "director"
        """
        stop_words = {"extract", "find", "get", "the", "a", "an", "of", "value", "text", "name", "movie", "page"}
        words = re.findall(r'\w+', intent.lower())

        # Filter words
        candidates = [w for w in words if w not in stop_words]

        # Return the last meaningful word (usually the subject)
        return candidates[-1] if candidates else words[-1]

    def _smart_clean_html(self, html_content, target_keyword=None):
        soup = BeautifulSoup(html_content, 'html.parser')

        # Aggressive cleaning
        for element in soup(["script", "style", "footer", "nav", "header", "svg", "button", "meta", "noscript", "iframe", "ad"]):
            element.decompose()

        text_structure = soup.get_text(separator="\n", strip=True)
        body_text = str(soup.body)

        # FOCUS LOGIC: Smaller window to prevent distraction
        if target_keyword:
            idx = body_text.lower().find(target_keyword.lower())
            if idx != -1:
                start = max(0, idx - 500)
                end = min(len(body_text), idx + 1500)
                return f"...{body_text[start:end]}...", text_structure

        return body_text[:4000], text_structure

    def _generate_selector(self, html_snippet, intent):
        prompt = f"""### Instruction:
        You are a DOM-aware agent. Analyze the HTML and return the CSS selector for: {intent}

        ### Input:
        {html_snippet}

        ### Response:
        """
        output = self.llm(prompt, max_tokens=64, stop=["<|im_end|>", "\n"], echo=False, temperature=0.1)
        return output['choices'][0]['text'].strip()

    def _direct_extraction(self, text_content, intent):
        Added "Negative Constraints" to stop it from grabbing actors
        prompt = f"""### Instruction:
        Extract the exact answer for the user request from the Text Context below.
        - Return ONLY the requested entity.
        - If asked for a Director, do NOT return Actors or Stars.
        - If asked for a Price, do NOT return the tax or savings.

        Request: {intent}

        Text Context:
        {text_content}

        ### Response:
        """
        output = self.llm(prompt, max_tokens=128, stop=["<|im_end|>", "\n"], echo=False, temperature=0.1)
        return output['choices'][0]['text'].strip()

    async def run_scrape(self, url, intent, broken_selector_simulation=False):
        status_log = st.empty()

        with status_log.status("🌐 initializing Crawler...", expanded=True) as s:
            async with AsyncWebCrawler(verbose=False) as crawler:
                s.write(f"Connecting to {url}...")
                result = await crawler.arun(url=url)

                if not result.success:
                    s.update(label="❌ Failed to load page", state="error")
                    return None

                soup = BeautifulSoup(result.html, 'html.parser')
                cached_selector = ".old-broken-selector" if broken_selector_simulation else None

                # STRATEGY 1: CACHED SELECTOR
                if cached_selector:
                    s.write(f"⚡ Testing Cached Selector: `{cached_selector}`")
                    await asyncio.sleep(0.5)
                    element = soup.select_one(cached_selector)
                    if element:
                        s.update(label="✅ Cache Hit!", state="complete")
                        return {"data": element.get_text(strip=True), "method": "Cache"}
                    else:
                        s.write("❌ Cache Failed. Engaging AI...")

                s.write("🧠 Analyzing Page Structure...")

                # V4: Better Subject Extraction
                keyword_hint = self._extract_subject(intent)
                s.write(f"🎯 Target Subject identified: `{keyword_hint}`")

                # Tighter Context Window
                html_context, text_structure = self._smart_clean_html(result.html, target_keyword=keyword_hint)

                # STRATEGY 2: GENERATE NEW SELECTOR
                new_selector = self._generate_selector(html_context, intent)
                s.write(f"💡 AI Suggested Selector: `{new_selector}`")

                try:
                    element = soup.select_one(new_selector)
                    if element and len(element.get_text(strip=True)) > 0:
                        data = element.get_text(strip=True)
                        s.update(label=f"✅ HEALED (Via Code)! Extracted: {data}", state="complete")
                        return {"data": data, "selector": new_selector, "method": "AI Selector Gen"}
                except:
                    pass

                # STRATEGY 3: DIRECT EXTRACTION (FALLBACK)
                s.write("🛡️ Engaging Fallback: Direct Text Extraction...")

                # Focus the text search strictly around the keyword
                keyword_idx = text_structure.lower().find(keyword_hint.lower())
                # Reduced window from 2000 to 800 chars to exclude "Stars" section
                focused_text = text_structure[max(0, keyword_idx-200):min(len(text_structure), keyword_idx+800)]

                direct_answer = self._direct_extraction(focused_text, intent)

                if direct_answer:
                    s.update(label=f"✅ HEALED (Via Text)! Extracted: {direct_answer}", state="complete")
                    return {"data": direct_answer, "selector": "N/A", "method": "Direct Text Scan"}
                else:
                    s.update(label="❌ All methods failed.", state="error")
                    return None

# UI
st.title("🚑 Self-Healing Scraper V4 (Production)")
col1, col2 = st.columns([1, 2])

with col1:
    url = st.text_input("Target URL", "https://www.imdb.com/title/tt0111161/")
    intent = st.text_input("What to extract?", "Extract the movie director name")
    simulate_break = st.checkbox("Simulate Broken Scraper?", value=True)

    if st.button("🚀 Launch Agent", type="primary"):
        agent = StreamlitAgent()
        result = asyncio.run(agent.run_scrape(url, intent, simulate_break))
        if result:
            st.success("Extraction Complete!")
            st.json(result)

with col2:
    st.info("ℹ️ V4 Update: Tighter Context Windows + Negative Prompting to prevent 'Actor' hallucinations.")

Overwriting app.py


In [ ]:
# STREAMLIT + NGROK DEPLOYMENT

import os

# UNCOMMENT THIS SECTION IF THE CELLS ABOVE CRASHED AND YOU JUST WANT TO RUN THE APP
# Note: You can actually make a simple script that doesn't need an ngrok token and just run it locally.

# 1. INSTALL NGROK & STREAMLIT
# !pip install -q pyngrok streamlit crawl4ai
# !pip install -q llama-cpp-python crawl4ai beautifulsoup4
# !playwright install
# !playwright install-deps

# 2. AUTHENTICATE
# ---------------------------------------------------------
# Replace the string below with your actual token from ngrok
# ---------------------------------------------------------
NGROK_TOKEN = "your_ngrok_api"  # <--- PASTE HERE

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_TOKEN)

# 3. RUN STREAMLIT IN BACKGROUND
# We run it on port 8501
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

# 4. START THE TUNNEL
# This connects the cloud port 8501 to the public internet
try:
    # Close old tunnels if any exist
    tunnels = ngrok.get_tunnels()
    for t in tunnels:
        ngrok.disconnect(t.public_url)

    # Open new tunnel
    public_url = ngrok.connect(8501).public_url
    print("------------------------------------------------")
    print("🚀 APP IS LIVE!")
    print(f"👉 Click here: {public_url}")
    print("------------------------------------------------")
except Exception as e:
    print(f"❌ Error: {e}")

# Keep the cell running
import time
while True:
    time.sleep(100)

------------------------------------------------
🚀 APP IS LIVE!
👉 Click here: https://rindless-jenee-eloquently.ngrok-free.dev
------------------------------------------------


## Troubleshooting & Testing
**Objective:** Establish a systematic approach to diagnosing failures in the AI/Browser interface.

**1. Infrastructure Validation:**
* **Runtime Config:** Ensure GPU VRAM is sufficient (~12GB for Training, ~4GB for Inference).
* **Model Integrity:** Verify the GGUF hash matches the source artifact.
    * *Command:* `ollama list` or check `llama_cpp` load logs for "signature mismatch."
* **Tokenizer Compatibility:** If the model outputs gibberish, verify the `transformers` library version matches the specific Qwen BPE requirement (v4.57.1).

**2. Logic Debugging (The "Selector Drift"):**
* **Symptom:** AI generates a selector that returns `null`.
* **Diagnosis Steps:**
    1.  **Context Check:** Dump the `html_context` variable. Does it actually contain the target price/text? If not, adjust the `_smart_clean_html` window logic.
    2.  **RAG Fidelity:** Verify that the `LlamaIndex` retriever isn't retrieving the footer instead of the main content.
    3.  **Fallback Trigger:** Ensure the `try/except` block correctly hands off control to the `Direct Text Extraction` agent upon selector failure.

**3. Test Suite Strategy:**
* **Unit Tests:** Validate `_extract_subject` NLP logic against edge cases (e.g., "Extract the lead actor's bio").
* **Integration Tests:** Run the full `scrape_with_agent` loop against a known "Golden Set" of static pages (`books.toscrape.com`) to benchmark accuracy.

## ⚠️ Disclaimer & Research Mode

> **Note:** The training notebook and local script run in **'Research Mode'** (bypassing restrictions) to generate synthetic training data and test out the fine-tuned model.

* This tool is a Proof of Concept (PoC) for autonomous software architecture.
* It utilizes `undetected-chromedriver` to test capabilities against anti-bot systems (like StockX).
* Users are solely responsible for ensuring their scraping activities comply with local laws and the Terms of Service of target websites.